In [ ]:
%%writefile lattice_3x3_springs_play.py
from manim import *
import numpy as np

class Lattice3x3Springs(Scene):
    def construct(self):
        N = 3
        a = 1.2
        k = 35.0
        c = 0.4
        m = 1.0
        dt = 1/120

        coils = 8
        amp = 0.12
        end_cap_frac = 0.12

        steps_per_frame = 1  # raise for faster/more obvious motion

        def idx(i, j): return i * N + j
        fixed = {idx(0, 0), idx(0, N - 1)}

        x0 = np.zeros((N*N, 2), dtype=float)
        for i in range(N):
            for j in range(N):
                x0[idx(i, j)] = [j*a, (N-1-i)*a]
        x0 -= np.mean(x0, axis=0)

        springs = []
        for i in range(N):
            for j in range(N):
                if j + 1 < N: springs.append((idx(i, j), idx(i, j+1), a))
                if i + 1 < N: springs.append((idx(i, j), idx(i+1, j), a))

        pos = x0.copy()
        vel = np.zeros_like(pos)
        pos[idx(1, 1)] += [0.6, -0.45]  # big poke so it’s obvious

        def to3(p2): return np.array([p2[0], p2[1], 0.0])

        def spring_pts(p, q):
            d = q - p
            L = np.linalg.norm(d)
            if L < 1e-6:
                return [to3(p), to3(q)]
            u = d / L
            perp = np.array([-u[1], u[0]])
            cap = end_cap_frac * L
            core = max(L - 2*cap, 1e-6)

            pts = [p, p + u*cap]
            for n in range(coils*2):
                t = (n + 1) / (coils*2 + 1)
                base = p + u*(cap + t*core)
                sign = 1 if n % 2 == 0 else -1
                pts.append(base + sign*amp*perp)
            pts += [q - u*cap, q]
            return [to3(x) for x in pts]

        def step():
            nonlocal pos, vel
            F = -c * vel
            for i, j, L0 in springs:
                d = pos[j] - pos[i]
                L = np.linalg.norm(d)
                if L > 1e-9:
                    e = d / L
                    f = k * (L - L0) * e
                    F[i] += f
                    F[j] -= f
            acc = F / m
            for fidx in fixed:
                pos[fidx] = x0[fidx]
                vel[fidx] = 0.0
                acc[fidx] = 0.0
            vel += acc * dt
            pos += vel * dt

        # visuals
        title = Text("3×3 Mass–Spring Lattice", font_size=34).to_edge(UP)
        self.add(title)

        masses = VGroup(*[
            Dot(radius=0.10, color=ORANGE if n in fixed else WHITE)
            for n in range(N*N)
        ])
        springs_vg = VGroup(*[
            VMobject().set_stroke(width=4)
            for _ in springs
        ])
        self.add(springs_vg, masses)

        lattice = VGroup(springs_vg, masses)

        def update_all(_):
            for _ in range(steps_per_frame):
                step()

            for n, dot in enumerate(masses):
                dot.move_to(to3(pos[n]))

            for s_mob, (i, j, _) in zip(springs_vg, springs):
                s_mob.set_points_as_corners(spring_pts(pos[i], pos[j]))

        self.play(UpdateFromFunc(lattice, update_all), run_time=6, rate_func=linear)



In [ ]:
!manim -ql --fps 15 --disable_caching lattice_3x3_springs_play.py Lattice3x3Springs

In [ ]:
!ls -lh media/videos/lattice_3x3_springs_play/*/Lattice3x3Springs.mp4

In [ ]:
from IPython.display import Video
Video("media/videos/lattice_3x3_springs_play/480p15/Lattice3x3Springs.mp4", embed=True)

In [ ]:
cd /users/PAS2669/moreno309/osc_classes/PHYS_5300_OSU

In [ ]:
ls

In [ ]:
!git init

In [ ]:
with open(".gitignore", "w") as f:
    f.write(
        "media/\n"
        "__pycache__/\n"
        "*.mp4\n"
        "*.mov\n"
    )

In [ ]:
!git status
!git add .gitignore
!git commit -m "Add gitignore"

In [ ]:
!git config --global user.name "Alex Moreno"
!git config --global user.email "alex.moreno312@gmail.com"

In [ ]:
!git add working_animation.ipynb .gitignore

In [ ]:
!git status

In [ ]:
!git commit -m "Add working animation notebook"

In [ ]:
!git branch -m main

In [ ]:
!git remote add orgin https://github.com/arm3094/TheorMech-Final.git

In [ ]:
!git remote -v

In [ ]:
!git remote rename orgin origin

In [ ]:
!git remote -v

In [ ]:
!git remote remove orgin

In [ ]:
!git remote -v

In [ ]:
!git branch